In [1]:
import numpy
import scipy.special

In [2]:
#definição da classe Rede Neural
class NeuralNet:
    
    #inicializar a rede
    def __init__(self, inputN, hiddenN, outputN, learningR):
        
        #Configurar o numero de nós em cada camada
        self.inodes = inputN
        self.hnodes = hiddenN
        self.onodes = outputN
        
        #taxa de aprendizado
        self.lr = learningR
        
        #WEIGHTS
        #Random weights from a normal probability distribution centred on zero
        #standard deviation = 1/srqt(incoming links)
        
        #numpy.random.normal(mean, spread, shape)
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        #ACTIVATION FUNCTION
        self.activation_function = lambda x : scipy.special.expit(x)
        
        pass
    
    #treinar a rede
    def train(self, inputs_list, targets_list):
        #convertere para array2d numpy
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #PROPAGATION
        #signals hiddenl
        hi = numpy.dot(self.wih, inputs) #hidden inputs
        ho = self.activation_function(hi) #hidden outputs
        
        #signals outputL
        oi = numpy.dot(self.who, ho) #final inputs
        oo = self.activation_function(oi) #final outputs
        
        #BACKpropagation
        #ERRORS
        output_errors = targets - oo #error = (target - actual)
        hidden_errors = numpy.dot(self.who.T, output_errors) #ErrorsH = who.T dot errorsOut
        
        #UPDATING WEIGHTS
        self.who += self.lr * numpy.dot((output_errors * oo * (1.0 - oo)), numpy.transpose(ho))
        self.wih += self.lr * numpy.dot((hidden_errors * ho * (1.0 - ho)), numpy.transpose(inputs))
        pass
    
    #resposta depois de treinada
    def query(self, inputs_list):
        
        #Convertendo inputs para Array2d
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #INPUT --> HIDDEN
        #Hidden input calculation
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        #Hidden outputs (activation function)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #HIDDEN --> OUTPUT
        #signals into the final layer (output layer)
        output_inputs = numpy.dot(self.who, hidden_outputs)
        #signals out the final layer (output layer)
        output_outputs = self.activation_function(output_inputs)
        
        return output_outputs

In [4]:
#MNIST NUMBERS Creating instance of the NN
#INPUTS (Every pixel 28x28), hidden, output (10 possible numbers)
inputl = 784
hiddenl = 200
outl = 10

learningrate = 0.1

nn = NeuralNet(inputl, hiddenl, outl, learningrate)

In [35]:
#loading the data train
import pandas as pd
from google.colab import files
file = files.upload()
x_train = pd.read_csv("mnist_train.csv", header = None)

In [40]:
#PREPARING THE DATA and TRAINING THE NN
# epochs is the number of times the training dataset is used for training
epochs = 5
x_train = numpy.array(x_train)
for i in range(epochs):
  for record in x_train:
    inputs = (record[1:] / 255.0 * 0.99) + 0.01 #scaling the inputs
    target = numpy.zeros(outl) + 0.01 #criando o target 0.01-0.99
    target[int(record[0])] = 0.99 #target label

    nn.train(inputs, target)
    pass
  print("Finishing epoch:", i, end="\r")
  pass

In [42]:
#loading the data test original 10
file = files.upload()
x_test_10 = pd.read_csv("mnist_test_10.csv", header = None)

Saving mnist_test_10.csv to mnist_test_10.csv


In [47]:
#test test_10
scorecard = []
x_test_10 = numpy.array(x_test_10)
for record in x_test_10:
    inputs = (record[1:] / 255.00 * 0.99) + 0.01
    
    predicted = numpy.argmax(nn.query(inputs)) #predicted value
    #scoreboard
    if (predicted == int(float(record[0]))):
        scorecard.append(1)
    else:
        scorecard.append(0)
    pass


print(scorecard)
#calculating performance
scorecard_array = numpy.asarray(scorecard)
performance = scorecard_array.sum() / scorecard_array.size
print ("Performance mnist_test_10=",  performance)
print ("Error =",  1.0 - performance)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Performance mnist_test_10= 1.0
Error = 0.0


In [58]:
file = files.upload()
x_test = pd.read_csv("meu_dataset (1).csv", header = None)

In [59]:
#MY OWN HANDWRITING
scorecard2 = []
x_test = numpy.array(x_test)
labels = []
for record in x_test:
    inputs = (record[1:] / 255.00 * 0.99) + 0.01
    
    predicted = numpy.argmax(nn.query(inputs)) #predicted value
    #scoreboard
    labels.append(record[0])
    if (predicted == record[0]):
        scorecard2.append(1)
    else:
        scorecard2.append(0)
    pass

print(labels)
print(scorecard2)
#calculating performance
scorecard_array = numpy.asarray(scorecard)
performance = scorecard_array.sum() / scorecard_array.size
print ("Performance (my own handwriting)=",  performance)
print ("Error =",  1.0 - performance)

[3.0, 7.0, 8.0, 9.0, 9.0]
[1, 0, 1, 1, 1]
Performance (my own handwriting)= 0.8
Error = 0.19999999999999996
